# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [249]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [250]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [251]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [252]:
data.shape # returns rows, columns

(68584, 7)

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [253]:
grouped_qty = data.groupby(['CustomerID', 'ProductName'])['Quantity'].sum().reset_index()

grouped_qty

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [254]:
prod_by_cust = data.pivot_table(
    index = 'ProductName',
    columns = 'CustomerID',
    values = 'Quantity',
    fill_value = 0,
    aggfunc = 'sum',
).transpose()

prod_by_cust

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0,0,0,25,0,50,0,25,0,0,...,0,25,25,0,0,0,0,0,0,0
98069,0,0,0,25,0,25,0,0,0,25,...,0,0,0,0,0,0,0,0,0,0
98159,0,0,0,0,0,0,0,0,0,0,...,0,50,0,0,0,0,25,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [255]:
squareform(pdist(prod_by_cust, 'euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [256]:
distances = pd.DataFrame(1/(1 + squareform(pdist(prod_by_cust, 'euclidean'))))

distances.columns = distances.index = prod_by_cust.index

distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [257]:
# set customerid to a variable so you can iterate/dynamically update later
similarities = distances[200].sort_values(ascending=False)[1:6] # start at index one to skip correlation with itself
similarities

CustomerID
3317    0.084959
1072    0.083972
3535    0.082403
1920    0.082100
3909    0.080920
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [258]:
top5 = distances[200].sort_values(ascending=False)[1:6]
top5

CustomerID
3317    0.084959
1072    0.083972
3535    0.082403
1920    0.082100
3909    0.080920
Name: 200, dtype: float64

In [259]:
# for cust_id in top5.index: # loop through the index of top5 instead of the distance-values
#     print(cust_id)
#     print(grouped_qty[grouped_qty['CustomerID'] == cust_id])
    
top5_similars = [grouped_qty[grouped_qty['CustomerID'] == cust_id] for cust_id in top5.index]
top5_similars

[      CustomerID                      ProductName  Quantity
 2788        3317                         Bay Leaf         1
 2789        3317             Beef - Ground Medium         1
 2790        3317               Beef - Rib Eye Aaa         1
 2791        3317                Butter - Unsalted         1
 2792        3317      Cheese - Victor Et Berthold         1
 2793        3317                        Corn Meal         1
 2794        3317          Crab - Dungeness, Whole         1
 2795        3317          Crab - Imitation Flakes         1
 2796        3317               Crush - Cream Soda         1
 2797        3317     Cup - Translucent 7 Oz Clear         1
 2798        3317               Doilies - 5, Paper         1
 2799        3317            Ecolab - Solid Fusion         1
 2800        3317             Ezy Change Mophandle         1
 2801        3317                   Fenngreek Seed         1
 2802        3317             Garbage Bags - Clear         1
 2803        3317       

In [260]:
sim_customers = pd.DataFrame()

for i in [grouped_qty[grouped_qty['CustomerID'] == i] for i in top5.index]:
    sim_customers = sim_customers.append(i)
    
sim_customers

,CustomerID,ProductName,Quantity
2788,3317,Bay Leaf,1
2789,3317,Beef - Ground Medium,1
2790,3317,Beef - Rib Eye Aaa,1
2791,3317,Butter - Unsalted,1
2792,3317,Cheese - Victor Et Berthold,1
...,...,...,...
3263,3909,Wine - Charddonnay Errazuriz,2
3264,3909,Wine - Fume Blanc Fetzer,1
3265,3909,"Wine - Magnotta, Merlot Sr Vqa",1
3266,3909,Wine - Toasted Head,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [261]:
# sim_customers.groupby('ProductName')['Quantity'].sum().sort_values(ascending = False)
sim2 = sim_customers.groupby('ProductName')['Quantity'].sum().sort_values(ascending = False).reset_index()
sim2

,ProductName,Quantity
0,Soup - Campbells Bean Medley,4
1,Muffin - Carrot Individual Wrap,3
2,Bay Leaf,3
3,Pork - Kidney,3
4,"Pepper - Black, Whole",3
...,...,...
202,Otomegusa Dashi Konbu,1
203,"Oregano - Dry, Rubbed",1
204,Onions - Cippolini,1
205,Olives - Stuffed,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [262]:
# sim_customers.join(prod_by_cust, on = 'ProductName', how = 'outer')
# prod_by_cust.join(sim_customers, on = 'ProductName', how = 'outer') # key error 
# prod_by_cust.transpose()[prod_by_cust.transpose()['CustomerID'] == 200]


user = prod_by_cust.transpose()[200]

user

ProductName
Anchovy Paste - 56 G Tube            0
Appetizer - Mini Egg Roll, Shrimp    0
Appetizer - Mushroom Tart            0
Appetizer - Sausage Rolls            0
Apricots - Dried                     0
                                    ..
Yeast Dry - Fermipan                 0
Yoghurt Tubes                        0
Yogurt - Blueberry, 175 Gr           1
Yogurt - French Vanilla              0
Zucchini - Yellow                    0
Name: 200, Length: 452, dtype: int64

In [263]:
# to_buy = sim_customers.join(user, on = 'ProductName', how = 'left')
to_buy = sim2.join(user, on = 'ProductName', how = 'left')

to_buy

,ProductName,Quantity,200
0,Soup - Campbells Bean Medley,4,0
1,Muffin - Carrot Individual Wrap,3,0
2,Bay Leaf,3,0
3,Pork - Kidney,3,0
4,"Pepper - Black, Whole",3,1
...,...,...,...
202,Otomegusa Dashi Konbu,1,0
203,"Oregano - Dry, Rubbed",1,0
204,Onions - Cippolini,1,0
205,Olives - Stuffed,1,1


In [320]:
# np.where to filter?
# group by the products with a zero in the '200' column, and sort descendingly?

to_buy[to_buy[200] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'].values[0]

'Bay Leaf'

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [358]:
unique_custIDs = list(data['CustomerID'].unique())

def recommendProducts(df: pd.DataFrame, cust_ids: list, distance_func = 'euclidean') -> pd.DataFrame:
    
    distances = pd.DataFrame(1/(1 + squareform(pdist(prod_by_cust, distance_func))))

    distances.columns = distances.index = prod_by_cust.index

    product1 = []
    product2 = []
    product3 = []
    product4 = []
    product5 = []
    
    for cid in cust_ids:
        
        top5 = distances[cid].sort_values(ascending=False)[1:6]

        sim_customers = pd.DataFrame()
        sim_customers = sim_customers.append([grouped_qty[grouped_qty['CustomerID'] == i] for i in top5.index])

        one_customer = prod_by_cust.transpose()[cid]

        to_buy = sim_customers.join(one_customer, on = 'ProductName', how = 'left')
        
        product1.append(to_buy[to_buy[cid] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'].values[0])
        product2.append(to_buy[to_buy[cid] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'].values[1])
        product3.append(to_buy[to_buy[cid] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'].values[2])
        product4.append(to_buy[to_buy[cid] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'].values[3])
        product5.append(to_buy[to_buy[cid] == 0].sort_values('Quantity', ascending = False)[:5]['ProductName'].values[4])

    return pd.DataFrame({'CustomerID': cust_ids,
                         'Product 1':product1, 
                         'Product 2':product2,
                         'Product 3':product3,
                         'Product 4':product4,
                         'Product 5':product5})
    

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [342]:
results = recommendProducts(prod_by_cust, unique_custIDs, 'euclidean')
results.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Pasta - Orecchiette,Squid U5 - Thailand,Chicken - Soup Base,Dc Hikiage Hira Huba,Ezy Change Mophandle
1,77352,Cocktail Napkin Blue,Chicken - Soup Base,Cheese - Taleggio D.o.p.,Wine - Ruffino Chianti,Tuna - Salad Premix
2,40094,Sherry - Dry,Grouper - Fresh,Beef - Inside Round,Halibut - Steaks,"Oregano - Dry, Rubbed"
3,23548,Banana Turning,"Appetizer - Mini Egg Roll, Shrimp",Wine - Chardonnay South,Shrimp - 31/40,Soupcontfoam16oz 116con
4,78981,Lettuce - Frisee,Vanilla Beans,Pop Shoppe Cream Soda,Sausage - Liver,Beef - Inside Round


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [343]:
#euclidean
results = recommendProducts(prod_by_cust, unique_custIDs, 'euclidean')
results.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Pasta - Orecchiette,Squid U5 - Thailand,Chicken - Soup Base,Dc Hikiage Hira Huba,Ezy Change Mophandle
1,77352,Cocktail Napkin Blue,Chicken - Soup Base,Cheese - Taleggio D.o.p.,Wine - Ruffino Chianti,Tuna - Salad Premix
2,40094,Sherry - Dry,Grouper - Fresh,Beef - Inside Round,Halibut - Steaks,"Oregano - Dry, Rubbed"
3,23548,Banana Turning,"Appetizer - Mini Egg Roll, Shrimp",Wine - Chardonnay South,Shrimp - 31/40,Soupcontfoam16oz 116con
4,78981,Lettuce - Frisee,Vanilla Beans,Pop Shoppe Cream Soda,Sausage - Liver,Beef - Inside Round


In [344]:
#cityblock
results = recommendProducts(prod_by_cust, unique_custIDs, 'cityblock')
results.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Sardines,"Oranges - Navel, 72",Beans - Wax,Bay Leaf,Wine - Charddonnay Errazuriz
1,77352,Tia Maria,Wine - Vidal Icewine Magnotta,Soup - Campbells Tomato Ravioli,Muffin - Carrot Individual Wrap,"Pepper - White, Ground"
2,40094,Wine - Chardonnay South,Sardines,Knife Plastic - White,Butter - Unsalted,Muffin - Carrot Individual Wrap
3,23548,Ice Cream Bar - Hageen Daz To,Cod - Black Whole Fillet,Bar - Granola Trail Mix Fruit Nut,Butter - Unsalted,Beets - Mini Golden
4,78981,Lettuce - Spring Mix,Table Cloth 81x81 White,Butter - Unsalted,Bar - Granola Trail Mix Fruit Nut,Wine - Vidal Icewine Magnotta


In [345]:
#braycurtis
results = recommendProducts(prod_by_cust, unique_custIDs, 'braycurtis')
results.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Mustard - Seed,Ecolab - Mikroklene 4/4 L,"Cheese - Brie, Triple Creme",Water - Aquafina Vitamin,"Coconut - Shredded, Sweet"
1,77352,Flavouring - Orange,Tia Maria,Milk - 2%,Cheese - Cheddarsliced,Lettuce - Frisee
2,40094,Juice - V8 Splash,Beer - Sleemans Cream Ale,"Wine - White, Mosel Gold",Cornflakes,Coffee - Hazelnut Cream
3,23548,Sponge Cake Mix - Chocolate,Cheese Cloth No 100,Milk - 2%,"Lamb - Pieces, Diced","Cup - 6oz, Foam"
4,78981,Bread - Raisin Walnut Oval,Juice - V8 Splash,Tea - Decaf Lipton,Rum - Mount Gay Eclipes,Onion Powder


In [346]:
#canberra
results = recommendProducts(prod_by_cust, unique_custIDs, 'canberra')
results.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,"Wine - White, Schroder And Schyl",Lime Cordial - Roses,Pastry - Butterscotch Baked,Wine - White Cab Sauv.on,"Cheese - Brie, Triple Creme"
1,77352,Mangoes,Grenadine,Grenadine,Chinese Foods - Chicken,Beef - Prime Rib Aaa
2,40094,Fuji Apples,Sugar - Fine,Macaroons - Two Bite Choc,Cinnamon Buns Sticky,"Wine - Red, Colio Cabernet"
3,23548,"Sauce - Gravy, Au Jus, Mix",Beef - Texas Style Burger,Napkin White - Starched,Sage - Ground,"Appetizer - Mini Egg Roll, Shrimp"
4,78981,Cookie Dough - Double,Duck - Breast,V8 - Berry Blend,Muffin - Zero Transfat,Beef Wellington


In [347]:
#chebyshev
results = recommendProducts(prod_by_cust, unique_custIDs, 'chebyshev')
results.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Oil - Shortening - All - Purpose,Grenadine,Beef - Prime Rib Aaa,Ice Cream Bar - Hageen Daz To,Chinese Foods - Chicken
1,77352,Cumin - Whole,"Cheese - Brie,danish",Ocean Spray - Kiwi Strawberry,Ice Cream Bar - Drumstick,Wine - Vidal Icewine Magnotta
2,40094,Pants Custom Dry Clean,Bouq All Italian - Primerba,Wine - White Cab Sauv.on,Halibut - Fletches,Beef - Inside Round
3,23548,Oil - Safflower,Yoghurt Tubes,Scallops 60/80 Iqf,Grouper - Fresh,Snapple - Iced Tea Peach
4,78981,Ocean Spray - Kiwi Strawberry,Rum - Mount Gay Eclipes,"Tart Shells - Sweet, 4","Bar Mix - Pina Colada, 355 Ml",Grapes - Red


In [348]:
#correlation
results = recommendProducts(prod_by_cust, unique_custIDs, 'correlation')
results.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Wine - Two Oceans Cabernet,Jagermeister,Ecolab - Solid Fusion,Wonton Wrappers,Cinnamon Buns Sticky
1,77352,Fondant - Icing,"Potatoes - Instant, Mashed",Bread - Rye,Lettuce - Frisee,Mangoes
2,40094,Juice - V8 Splash,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Beef - Inside Round,"Lentils - Red, Dry"
3,23548,Wine - Charddonnay Errazuriz,Meldea Green Tea Liquor,Foam Cup 6 Oz,Assorted Desserts,"Coconut - Shredded, Sweet"
4,78981,Sauerkraut,"Cup - 6oz, Foam",Quiche Assorted,Rum - Mount Gay Eclipes,Grapes - Red


In [349]:
#cosine
results = recommendProducts(prod_by_cust, unique_custIDs, 'cosine')
results.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Wine - Two Oceans Cabernet,Jagermeister,Ecolab - Solid Fusion,Wonton Wrappers,Cinnamon Buns Sticky
1,77352,"Potatoes - Instant, Mashed",Fondant - Icing,Bread - Rye,Lettuce - Frisee,Beer - Rickards Red
2,40094,Juice - V8 Splash,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Beef - Inside Round,Coffee - Hazelnut Cream
3,23548,Bagel - Plain,Mussels - Cultivated,Whmis - Spray Bottle Trigger,"Salsify, Organic",Wine - Charddonnay Errazuriz
4,78981,Cocoa Butter,Wine - Blue Nun Qualitatswein,Beef - Top Sirloin - Aaa,Cheese - Cottage Cheese,Baking Powder


In [350]:
#jensenshannon
results = recommendProducts(prod_by_cust, unique_custIDs, 'jensenshannon')
results.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,"Tart Shells - Sweet, 4",Garbage Bags - Clear,"Lentils - Red, Dry",Ecolab - Solid Fusion,Tea - Jasmin Green
1,77352,Fondant - Icing,"Potatoes - Instant, Mashed",Bread - Rye,Lettuce - Frisee,Grenadine
2,40094,Juice - V8 Splash,Mayonnaise - Individual Pkg,Truffle Cups - Brown,Beef - Inside Round,"Bread - Roll, Soft White Round"
3,23548,Wine - Charddonnay Errazuriz,Assorted Desserts,Zucchini - Yellow,Olives - Stuffed,Bandage - Flexible Neon
4,78981,Rum - Mount Gay Eclipes,Tea - Decaf Lipton,Mussels - Frozen,Onion Powder,Table Cloth - 53x69 Colour


In [351]:
#mahalanobis
results = recommendProducts(prod_by_cust, unique_custIDs, 'mahalanobis')
results.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Cumin - Whole,Smirnoff Green Apple Twist,Ice Cream Bar - Drumstick,Cake - Mini Cheesecake,"Pepper - Black, Whole"
1,77352,Appetizer - Sausage Rolls,Bacardi Breezer - Tropical,Flavouring - Orange,Spinach - Baby,Crush - Cream Soda
2,40094,Watercress,Smirnoff Green Apple Twist,Lamb - Ground,Bread - Italian Corn Meal Poly,General Purpose Trigger
3,23548,"Cheese - Boursin, Garlic / Herbs","Rum - Coconut, Malibu","Doilies - 5, Paper",Ketchup - Tomato,"Lentils - Red, Dry"
4,78981,Beef - Ground Medium,Veal - Inside,Cod - Black Whole Fillet,"Bar Mix - Pina Colada, 355 Ml",Wine - Fume Blanc Fetzer


In [352]:
#minkowski
results = recommendProducts(prod_by_cust, unique_custIDs, 'minkowski')
results.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Pasta - Orecchiette,Squid U5 - Thailand,Chicken - Soup Base,Dc Hikiage Hira Huba,Ezy Change Mophandle
1,77352,Cocktail Napkin Blue,Chicken - Soup Base,Cheese - Taleggio D.o.p.,Wine - Ruffino Chianti,Tuna - Salad Premix
2,40094,Sherry - Dry,Grouper - Fresh,Beef - Inside Round,Halibut - Steaks,"Oregano - Dry, Rubbed"
3,23548,Banana Turning,"Appetizer - Mini Egg Roll, Shrimp",Wine - Chardonnay South,Shrimp - 31/40,Soupcontfoam16oz 116con
4,78981,Lettuce - Frisee,Vanilla Beans,Pop Shoppe Cream Soda,Sausage - Liver,Beef - Inside Round


In [353]:
#seuclidean
results = recommendProducts(prod_by_cust, unique_custIDs, 'seuclidean')
results.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Squid U5 - Thailand,Yoghurt Tubes,Pasta - Orecchiette,Bread - Multigrain,"Potatoes - Instant, Mashed"
1,77352,Cocktail Napkin Blue,Chicken - Soup Base,Cheese - Taleggio D.o.p.,Wine - Ruffino Chianti,Wiberg Super Cure
2,40094,Sherry - Dry,Grouper - Fresh,"Oregano - Dry, Rubbed",Halibut - Steaks,Beef - Inside Round
3,23548,Banana Turning,"Appetizer - Mini Egg Roll, Shrimp","Wine - Red, Colio Cabernet",Sprouts - Baby Pea Tendrils,Squid - Tubes / Tenticles 10/20
4,78981,Lettuce - Frisee,Vanilla Beans,Beef - Inside Round,Sausage - Liver,Pop Shoppe Cream Soda


In [354]:
#sqeuclidean
results = recommendProducts(prod_by_cust, unique_custIDs, 'sqeuclidean')
results.head()

,CustomerID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Pasta - Orecchiette,Squid U5 - Thailand,Chicken - Soup Base,Dc Hikiage Hira Huba,Ezy Change Mophandle
1,77352,Cocktail Napkin Blue,Chicken - Soup Base,Cheese - Taleggio D.o.p.,Wine - Ruffino Chianti,Tuna - Salad Premix
2,40094,Sherry - Dry,Grouper - Fresh,Beef - Inside Round,Halibut - Steaks,"Oregano - Dry, Rubbed"
3,23548,Banana Turning,"Appetizer - Mini Egg Roll, Shrimp",Wine - Chardonnay South,Shrimp - 31/40,Soupcontfoam16oz 116con
4,78981,Lettuce - Frisee,Vanilla Beans,Pop Shoppe Cream Soda,Sausage - Liver,Beef - Inside Round


In [356]:
#wminkowski
# results = recommendProducts(prod_by_cust, unique_custIDs, 'wminkowski')
# results.head()